In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import randint

from numpy import mean
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV
#from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from ipynb.fs.full.preprocessing import preprocessing #import preprocessing class from preprocessing.ipynb

In [2]:
#option 1: call preprocessing class to generate dataframes
prep_dwt = preprocessing(file_path = '/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/data/DWT.csv') #initialize class for DWT_8702
dwt = prep_dwt.label_df() #generate dataframe with labels

prep_wang = preprocessing(file_path = '/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/data/Wang.csv') #initialize class for WPD_Wang_8702
wang = prep_wang.label_df()

prep_jones = preprocessing(file_path = '/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/data/Jones.csv') #initialize class for WPD_Wang_8702
jones = prep_jones.label_df()


In [11]:
def r_forest_default(df, n_iter = 1000):
    df = df

    subset_0 = df[df['state'] == 0]
    subset_1 = df[df['state'] == 1]

    subset_1 = subset_1.sample(n=91) #randomly sample cases to match number of controls
    df = pd.concat([subset_0, subset_1])

    x = df.loc[:, df.columns != 'state'] #features
    y = df.loc[:, df.columns == 'state'] #supervisor

    eval_metrics = { #empty dictionary to store classification report
        '0_precision': [],
        '0_recall': [],
        '0_f1-score': [],
        '0_support': [],

        '1_precision': [],
        '1_recall': [],
        '1_f1-score': [],
        '1_support': [],
        
        'accuracy': [],
        'accuracy_train': [],
    }
    
    for i in range(n_iter): 

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.67)
        
        tree_model = RandomForestClassifier()
        tree_model.fit(x_train, y_train.values.ravel())
        y_pred = tree_model.predict(x_test)

        report = classification_report(y_test, y_pred, output_dict=True)

        y_pred_train = tree_model.predict(x_train)
        report_train = classification_report(y_train, y_pred_train, output_dict=True)

        eval_metrics['0_precision'].append(report['0']['precision']) #store classification_report values in dictionary
        eval_metrics['0_recall'].append(report['0']['recall'])
        eval_metrics['0_f1-score'].append(report['0']['f1-score'])
        eval_metrics['0_support'].append(report['0']['support'])

        eval_metrics['1_precision'].append(report['1']['precision'])
        eval_metrics['1_recall'].append(report['1']['recall'])
        eval_metrics['1_f1-score'].append(report['1']['f1-score'])
        eval_metrics['1_support'].append(report['1']['support'])

        eval_metrics['accuracy'].append(report['accuracy'])
        eval_metrics['accuracy_train'].append(report_train['accuracy'])

    eval_metrics_df = pd.DataFrame.from_dict(eval_metrics) #convert dictionary to dataframe

    return eval_metrics_df

In [13]:
acc_df_dwt = r_forest_default(dwt, n_iter=1000)
acc_df_wang = r_forest_default(wang, n_iter=1000)
acc_df_jones = r_forest_default(jones, n_iter=1000)

print('DWT Method Accuracy ', round(acc_df_dwt['accuracy'].mean(), 4))
print('Wang Method Accuracy ', round(acc_df_wang['accuracy'].mean(), 4))
print('Jones Method Accuracy ', round(acc_df_jones['accuracy'].mean(), 4))

DWT Method Accuracy  0.9198
Wang Method Accuracy  0.9303
Jones Method Accuracy  0.8094


In [24]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data

df = wp8
x = df.loc[:, df.columns != 'state'] #features
y = df.loc[:, df.columns == 'state'] #supervisor

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.67)
    
rand_search.fit(x_train, y_train.values.ravel())

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x166fcd850>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x166e864f0>})

In [25]:
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'max_depth': 15, 'n_estimators': 71}


In [28]:
y_pred = best_rf.predict(x_test)

acc = classification_report(y_test, y_pred, output_dict=True)['accuracy']
spec = classification_report(y_test, y_pred, output_dict=True)['1']['precision'] #specificity
sens = classification_report(y_test, y_pred, output_dict=True)['1']['recall'] #sensitivity

print(acc)


0.9285714285714286 0.9491525423728814 0.9491525423728814


In [35]:


# Fit the random search object to the data

df = wp8
x = df.loc[:, df.columns != 'state'] #features
y = df.loc[:, df.columns == 'state'] #supervisor

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.67)





param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

    
rand_search.fit(x_train, y_train.values.ravel())

best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

y_pred = best_rf.predict(x_test)

acc = classification_report(y_test, y_pred, output_dict=True)['accuracy']
spec = classification_report(y_test, y_pred, output_dict=True)['1']['precision'] #specificity
sens = classification_report(y_test, y_pred, output_dict=True)['1']['recall'] #sensitivity

print(acc)


Best hyperparameters: {'max_depth': 9, 'n_estimators': 417}
0.9642857142857143


In [36]:
def tree_rep_samples(df, n_iter = 1000):
    df = df

    subset_0 = df[df['state'] == 0]
    subset_1 = df[df['state'] == 1]

    subset_1 = subset_1.sample(n=91) #randomly sample cases to match number of controls
    df = pd.concat([subset_0, subset_1])
    
    x = df.loc[:, df.columns != 'state'] #features
    y = df.loc[:, df.columns == 'state'] #supervisor

    eval_metrics = {} #empty dictionary to store metrics

    for i in range(n_iter): 

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.67)

        param_dist = {'n_estimators': randint(50,500),
                    'max_depth': randint(1,20)}

        # Create a random forest classifier
        rf = RandomForestClassifier()

        # Use random search to find the best hyperparameters
        rand_search = RandomizedSearchCV(rf, 
                                        param_distributions = param_dist, 
                                        n_iter=5, 
                                        cv=5)
            
        rand_search.fit(x_train, y_train.values.ravel())

        best_rf = rand_search.best_estimator_

        y_pred = best_rf.predict(x_test)

        acc = classification_report(y_test, y_pred, output_dict=True)['accuracy']
        spec = classification_report(y_test, y_pred, output_dict=True)['1']['precision'] #specificity
        sens = classification_report(y_test, y_pred, output_dict=True)['1']['recall'] #sensitivity

        metrics = [acc, spec, sens] #store values in list
        eval_metrics[i]=list(metrics) #store list in dictionary

    eval_metrics = pd.DataFrame.from_dict(eval_metrics).T 
    eval_metrics.columns = ['accuracy', 'specificity', 'sensitivity']
    eval_metrics.head()

    means = eval_metrics.mean(axis=0) #mean of each column
    
    return means

In [38]:
tree_rep_samples(dwt8, n_iter = 100)

accuracy       0.911190
specificity    0.900423
sensitivity    0.968746
dtype: float64

In [39]:
tree_rep_samples(wp8, n_iter=100)

accuracy       0.936310
specificity    0.930825
sensitivity    0.973623
dtype: float64

In [43]:
def tree_rep_samples(df, n_iter = 1000):
    df = df
    x = df.loc[:, df.columns != 'state'] #features
    y = df.loc[:, df.columns == 'state'] #supervisor

    eval_metrics = {} #empty dictionary to store metrics

    for i in range(n_iter): 

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.67)

        param_grid = { 
            'n_estimators': [200, 500],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_depth' : [4,5,6,7,8],
            'criterion' :['gini', 'entropy']
            }       

        # Create a random forest classifier
        rf = RandomForestClassifier()

        # Use random search to find the best hyperparameters
        rand_search = RandomizedSearchCV(rf, 
                                        param_distributions = param_grid, 
                                        n_iter=10, 
                                        cv=5)
            
        rand_search.fit(x_train, y_train.values.ravel())

        best_rf = rand_search.best_estimator_

        y_pred = best_rf.predict(x_test)

        acc = classification_report(y_test, y_pred, output_dict=True)['accuracy']
        spec = classification_report(y_test, y_pred, output_dict=True)['1']['precision'] #specificity
        sens = classification_report(y_test, y_pred, output_dict=True)['1']['recall'] #sensitivity

        metrics = [acc, spec, sens] #store values in list
        eval_metrics[i]=list(metrics) #store list in dictionary

    eval_metrics = pd.DataFrame.from_dict(eval_metrics).T 
    eval_metrics.columns = ['accuracy', 'specificity', 'sensitivity']
    eval_metrics.head()

    means = eval_metrics.mean(axis=0) #mean of each column
    
    return means

In [46]:
tree_rep_samples(dwt8, n_iter = 50)

/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
35 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/.venv/lib/python3.9/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/jhbyun/Document

accuracy       0.913333
specificity    0.903434
sensitivity    0.971320
dtype: float64

In [47]:
tree_rep_samples(wp8, n_iter = 50)

/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jhbyun/Documents/685-Pr/ovarian-cancer-classification/.venv/lib/python3.9/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/jhbyun/Document

accuracy       0.932381
specificity    0.926212
sensitivity    0.973543
dtype: float64